<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.62
    Uninstalling yfinance-0.2.62:
      Successfully uninstalled yfinance-0.2.62
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-06-12 11:19:56
-------------------
qualified stocks: 90
with latest results: 29
still star stocks: 17
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  5.50 L
Current:  1.36 C
-------------------
Today PnL: -13.92 K (-0.1%)
Current PnL: -14.26 L (-9.94%)
CY Booked + Current PnL: -7.89 L (-5.5%)
-------------------
Total profit:  3.58 L
Total loss:  -17.84 L
-------------------
Total Booked + Current PnL: 19.09 L (16.38%)
Total Booked PnL: 33.35 L (28.62%)
Curr Year Booked PnL: 6.37 L (4.69%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.05 C
Est FTT PnL: 69.45 L (51.18%)
Deployed:  1.17 C
Current:  1.36 C
CAGR/XIRR %: 10.75%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,LICI,827.06,983.0,-2.96,H-LC,85.56,98957.0,13770.0,2296.0,-0.42,16.16,2.32,18.85,34.0,6.00,0.76,31.75,XY25,NTT,INSURANCE
59,RELIANCE,1291.83,1526.0,6.50,X-LC,26.67,158039.0,17230.0,8297.0,0.07,12.24,5.25,18.13,19.0,2.08,1.21,24.79,XY25,NTT,REFINERIES
3,ADANIPORTS,1103.69,1583.0,2.87,M-LC,53.33,103021.0,24659.0,9375.0,-0.37,31.47,9.10,43.43,77.0,2.63,0.79,37.79,XY24,NTT,MISC
81,UJJIVANSFB,52.77,53.0,54.03,M-SC,81.11,133272.0,-9207.0,9822.0,0.71,-6.46,7.37,0.44,241.0,-0.94,1.02,57.30,OX40N,NTT,BANKS
75,TATAELXSI,7332.28,7332.0,-12.62,X-MC,57.78,91672.0,-10980.0,10973.0,-1.15,-10.70,11.97,-0.00,55.0,-1.00,0.70,39.35,OX40N,NTT,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,NATIONALUM,189.63,247.44,-37.21,L-SC,68.89,99990.0,-514.0,31157.0,-0.71,-0.51,31.16,30.49,246.0,-0.02,0.77,31.73,X5K,ATH,METALS
13,BERGEPAINT,561.33,680.00,-16.80,H-MC,44.44,228015.0,676.0,47382.0,-0.74,0.30,20.78,21.14,118.0,0.01,1.75,27.66,XY24,NTT,PAINTS
30,HAVELLS,1588.02,2069.16,-4.90,X-LC,15.56,245840.0,-1891.0,76948.0,-0.20,-0.76,31.30,30.30,50.0,-0.02,1.89,12.43,X40,ATH,ELECTRICAL


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,ROUTE,1558.40,2249.26,-29.28,M-SC,93.33,76048.0,-26806.0,72405.0,12.19,-26.06,95.21,44.33,187.0,-0.37,0.58,26.29,SR,ATH,IT
74,TANLA,917.30,1963.11,-27.77,H-SC,97.78,177441.0,-61974.0,334938.0,9.19,-25.89,188.76,114.01,127.0,-0.19,1.36,62.74,AR,ATH,IT
84,VALIANTORG,512.64,838.00,-405.05,H-SC,100.00,129779.0,-3507.0,88107.0,3.20,-2.63,67.89,63.47,144.0,-0.04,1.00,117.54,XR,NTT,CHEMICALS
6,ASIANPAINT,2961.56,4250.00,-23.13,X-LC,7.78,152952.0,-48434.0,136051.0,1.83,-24.05,88.95,43.51,5.0,-0.36,1.17,6.47,X40,BTT,PAINTS
66,SFL,1016.35,1287.00,287.26,H-SC,38.89,174743.0,-87475.0,157304.0,1.76,-33.36,90.02,26.63,146.0,-0.56,1.34,8.65,XY24,NTT,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,GREENPANEL,375.16,537.0,200.39,M-SC,86.67,134029.0,-49049.0,128025.0,-2.31,-26.79,95.52,43.14,210.0,-0.38,1.03,26.07,XY24,NTT,MISC
56,QUESS,711.66,986.0,NaN,NaN,1.11,67616.0,-88238.0,148316.0,-2.29,-56.62,219.35,38.55,190.0,-0.59,0.52,3.54,XY24,NTT,MISC
21,COFFEEDAY,59.14,80.0,-51.71,L-SC,92.22,70118.0,-43431.0,83482.0,-2.17,-38.25,119.06,35.27,260.0,-0.52,0.54,71.62,XR,NTT,HOTELS
63,SAMMAANCAP,170.35,326.0,-162.93,M-SC,52.22,76650.0,-25560.0,118953.0,-1.84,-25.01,155.19,91.37,199.0,-0.21,0.59,24.43,XY25,NTT,FINANCE
9,AWL,325.39,485.0,-68.28,H-MC,42.22,223616.0,-44831.0,176500.0,-1.79,-16.70,78.93,49.05,99.0,-0.25,1.72,14.97,XY24,NTT,FMCG


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UJJIVANSFB,52.77,53.0,54.03,M-SC,81.11,133272.0,-9207.0,9822.0,0.71,-6.46,7.37,0.44,241.0,-0.94,1.02,57.30,OX40N,NTT,BANKS
73,SYMPHONY,1306.42,1306.0,-27.28,M-SC,3.33,154973.0,-16168.0,16117.0,-0.57,-9.45,10.40,-0.03,189.0,-1.00,1.19,11.38,OX40N,NTT,DURABLES
75,TATAELXSI,7332.28,7332.0,-12.62,X-MC,57.78,91672.0,-10980.0,10973.0,-1.15,-10.70,11.97,-0.00,55.0,-1.00,0.70,39.35,OX40N,NTT,IT
86,VIPIND,488.80,489.0,-948.40,H-SC,88.89,80798.0,-14518.0,14560.0,0.18,-15.23,18.02,0.04,151.0,-1.00,0.62,59.37,OX40N,NTT,MISC
47,KANSAINER,299.63,340.0,-67.49,H-SC,8.89,226710.0,-42957.0,79280.0,0.44,-15.93,34.97,13.47,143.0,-0.54,1.74,13.47,XY24,NTT,PAINTS


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TCS,3794.03,4998.00,-17.44,X-LC,6.67,269233.0,-26701.0,120616.0,-0.58,-9.02,44.80,31.73,1.0,-0.22,2.07,7.76,X40,BTT,IT
42,INFY,1461.46,2275.00,-9.55,X-LC,25.56,289783.0,28182.0,117449.0,-0.75,10.77,40.53,55.67,2.0,0.24,2.23,17.54,X40,BTT,IT
51,LTIM,5564.16,7197.33,1.57,X-LC,64.44,195390.0,-4920.0,63717.0,0.62,-2.46,32.61,29.35,3.0,-0.08,1.50,34.99,X200,ATH,IT
6,ASIANPAINT,2961.56,4250.00,-23.13,X-LC,7.78,152952.0,-48434.0,136051.0,1.83,-24.05,88.95,43.51,5.0,-0.36,1.17,6.47,X40,BTT,PAINTS
34,HINDUNILVR,2413.81,2723.00,-15.98,X-LC,13.33,263906.0,-6441.0,41064.0,-0.85,-2.38,15.56,12.81,6.0,-0.16,2.03,9.22,XY25,NTT,FMCG


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,UNIONBANK,123.87,163.0,-1.84,M-LC,87.78,173120.0,32280.0,12205.0,-0.37,22.92,7.05,31.59,88.0,2.64,1.33,50.38,XY24,NTT,BANKS
3,ADANIPORTS,1103.69,1583.0,2.87,M-LC,53.33,103021.0,24659.0,9375.0,-0.37,31.47,9.10,43.43,77.0,2.63,0.79,37.79,XY24,NTT,MISC


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,130.64,228.00,101.81,M-MC,72.78,229198.0,4236.0,163418.0,-0.04,1.88,71.30,74.53,183.0,0.03,1.76,33.07,XY24,BTT,STEEL
35,HINDZINC,514.80,744.74,40.53,M-LC,90.00,105300.0,2340.0,43647.0,1.14,2.27,41.45,44.67,68.0,0.05,0.81,35.89,X5K,ATH,METALS
17,CAMS,3643.00,5250.99,3.56,H-SC,76.67,116340.0,14336.0,30690.0,-1.75,14.05,26.38,44.14,125.0,0.47,0.89,32.58,X40N,ATH,MISC
3,ADANIPORTS,1103.69,1583.00,2.87,M-LC,53.33,103021.0,24659.0,9375.0,-0.37,31.47,9.10,43.43,77.0,2.63,0.79,37.79,XY24,NTT,MISC
32,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,61.11,153783.0,2870.0,57576.0,-0.37,1.90,37.44,40.06,101.0,0.05,1.18,25.79,AR,ATH,AUTO


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
88,WHIRLPOOL,1167.49,2270.00,-31.00,M-SC,55.56,201287.0,29666.0,132407.0,1.47,17.29,65.78,94.43,191.0,0.22,1.55,48.56,XR,NTT,DURABLES
89,WIPRO,243.46,420.00,-5.63,M-LC,32.22,161690.0,10745.0,98712.0,0.72,7.12,61.05,72.51,70.0,0.11,1.24,11.54,XR,NTT,IT
11,BANKINDIA,116.91,190.00,-28.22,M-MC,75.56,191050.0,11242.0,101161.0,0.22,6.25,52.95,62.52,168.0,0.11,1.47,37.49,XR,NTT,BANKS
39,INDIAMART,2327.09,4871.06,-49.41,H-SC,58.89,131885.0,8549.0,126280.0,-0.39,6.93,95.75,109.32,122.0,0.07,1.01,30.29,AR,ATH,MISC
33,HINDALCO,651.95,761.55,-11.72,M-LC,27.78,105360.0,1048.0,16489.0,0.50,1.00,15.65,16.81,71.0,0.06,0.81,17.21,X5K,ATH,METALS


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,59.14,80.00,-51.71,L-SC,92.22,70118.0,-43431.0,83482.0,-2.17,-38.25,119.06,35.27,260.0,-0.52,0.54,71.62,XR,NTT,HOTELS
4,ALKYLAMINE,2347.98,4567.52,-7.21,X-SC,84.44,86383.0,-14580.0,110017.0,-0.39,-14.44,127.36,94.53,59.0,-0.13,0.66,28.07,SR,ATH,CHEMICALS
52,MASFIN,326.60,399.50,-20.96,H-SC,66.67,89670.0,-8310.0,30183.0,-0.83,-8.48,33.66,22.32,138.0,-0.28,0.69,30.05,XR,ATH,FINANCE
50,LICI,827.06,983.00,-2.96,H-LC,85.56,98957.0,13770.0,2296.0,-0.42,16.16,2.32,18.85,34.0,6.00,0.76,31.75,XY25,NTT,INSURANCE
3,ADANIPORTS,1103.69,1583.00,2.87,M-LC,53.33,103021.0,24659.0,9375.0,-0.37,31.47,9.10,43.43,77.0,2.63,0.79,37.79,XY24,NTT,MISC


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VALIANTORG,512.64,838.00,-405.05,H-SC,100.00,129779.0,-3507.0,88107.0,3.20,-2.63,67.89,63.47,144.0,-0.04,1.00,117.54,XR,NTT,CHEMICALS
7,ASIANTILES,75.41,137.00,6877.78,M-SC,98.89,76568.0,-17242.0,93857.0,-0.93,-18.38,122.58,81.67,235.0,-0.18,0.59,48.28,XR,NTT,CERAMICS
74,TANLA,917.30,1963.11,-27.77,H-SC,97.78,177441.0,-61974.0,334938.0,9.19,-25.89,188.76,114.01,127.0,-0.19,1.36,62.74,AR,ATH,IT
46,JPPOWER,18.73,26.20,-33.07,L-SC,96.67,135708.0,-6134.0,62711.0,-1.65,-4.32,46.21,39.88,256.0,-0.10,1.04,37.53,XY24,NTT,POWER
78,TITAGARH,1097.23,1548.00,-7.32,H-SC,95.56,162855.0,-29160.0,108038.0,-0.61,-15.19,66.34,41.08,152.0,-0.27,1.25,37.84,XY24,NTT,ENGINEERING


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.97
1,25,43.63
2,50,72.51


In [21]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     32.56
XY25     14.49
XR       11.24
X40      11.14
OX40N     8.62
X40N      8.57
AR        8.22
X5K       2.39
X200      1.50
SR        1.24
Name: CurrAlloc%, dtype: float64

In [22]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    19.35
X-LC    18.43
M-SC    14.63
M-LC    10.29
H-MC     9.08
H-LC     6.93
X-MC     6.27
M-MC     5.93
L-SC     3.30
X-SC     2.84
L-LC     1.14
Name: CurrAlloc%, dtype: float64

In [23]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.29,-1.71,35.56
IT,11.55,-10.36,66.39
BANKS,8.73,-7.18,50.37
MISC,6.68,-24.08,72.60
FINANCE,6.52,-18.45,64.03
PAINTS,5.73,-16.76,39.77
ELECTRICAL,5.24,-3.16,47.90
HEALTHCARE,4.83,-2.42,32.34
AUTO,4.79,-8.30,55.92


In [24]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2692737.0,25
XR,1099216.0,13
AR,998888.0,9
X40,598397.0,8
XY25,501751.0,10
X40N,412743.0,8
OX40N,303935.0,11
SR,182422.0,2
X5K,91293.0,3


In [25]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1843412.0,18
M-SC,1420897.0,18
X-LC,813401.0,11
M-MC,572254.0,5
H-MC,545395.0,7
M-LC,376663.0,8
X-MC,329073.0,5
L-SC,298639.0,5
X-SC,262083.0,4


In [26]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,883189.0,6
M-SC,XY24,569219.0,5
H-SC,AR,514999.0,3
X-LC,X40,451064.0,4
M-MC,XY24,386769.0,3
M-SC,XR,340354.0,4
H-MC,XY24,223882.0,2
M-LC,XY24,204748.0,4
H-SC,XR,190024.0,3


In [27]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
